<a href="https://colab.research.google.com/github/Upeshjeengar/Graph-Neural-Nets/blob/main/Graph_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch torch torchvision torchaudio torch-geometric matplotlib scikit-learn

In [9]:
#Necessary Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch_geometric.nn.conv import GCNConv

In [6]:
#Dataset
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root=".", name="Cora")
data = dataset[0]
num_labels = len(set(data.y.numpy()))  # used for output_dim

**About Dataset**    
* The Cora dataset is a benchmark dataset for graph neural networks. The dataset contains data about 2708 scientific publications. These publications are the nodes of the graph.     
* The target is to predict the subject of each paper, there are seven classes in total.

#Directly Using Neural Network


In [7]:
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.lin1 = nn.Linear(input_dim, hidden_dim)
        self.lin2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x = data.x  # no graph structure, only node features
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return F.log_softmax(x, dim=1)

In [18]:
def accuracy(y_pred,y):
  return accuracy_score(y, y_pred)*100

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = {}

# iterate over the different model types
for model_class in [MLP]: # later we test also with GCN (this post) and GAT (next blog post)
    results[model_class.__name__] = []
    for i in range(10):
        print(f"Training {model_class.__name__} iteration {i+1}")

        # the output_dim is the number of unique classes in the set
        model = model_class(input_dim=data.x.shape[1], hidden_dim=32, output_dim=num_labels).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

        # deal with the class imbalance
        class_weights = torch.bincount(data.y) / len(data.y)
        loss_fn = nn.CrossEntropyLoss(weight=1/class_weights).to(device)

        data = data.to(device)

        # training loop
        for epoch in range(100):
            model.train()
            optimizer.zero_grad()
            out = model(data)

            # calculate loss
            train_loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            train_loss.backward()
            optimizer.step()

            if epoch % 10 == 0:
                model.eval()
                with torch.no_grad():
                    val_loss = loss_fn(out[data.val_mask], data.y[data.val_mask])
                    val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                    print(f'Epoch {epoch} | Training Loss: {train_loss.item():.2f} | Train Acc: {acc:>5.2f} | Validation Loss: {val_loss.item():.2f} | Validation Acc: {val_acc:>5.2f}')

        # final evaluation on the test set
        model.eval()
        with torch.no_grad():
            out = model(data)
            test_loss = loss_fn(out[data.test_mask], data.y[data.test_mask])
            test_acc = accuracy(out[data.test_mask].argmax(dim=1), data.y[data.test_mask])
            print(f'{model_class.__name__} Test Loss: {test_loss.item():.2f} | Test Acc: {test_acc:>5.2f}')
            results[model_class.__name__].append([acc, val_acc, test_acc])


# print average on test set and standard deviation
for model_name, model_results in results.items():
    model_results = torch.tensor(model_results)
    print(f'{model_name} Test Accuracy: {model_results[:, 2].mean():.2f} ± {model_results[:, 2].std():.2f}')

Training MLP iteration 1
Epoch 0 | Training Loss: 1.94 | Train Acc: 12.14 | Validation Loss: 1.95 | Validation Acc:  8.60
Epoch 10 | Training Loss: 0.32 | Train Acc: 99.29 | Validation Loss: 1.56 | Validation Acc: 32.00
Epoch 20 | Training Loss: 0.03 | Train Acc: 100.00 | Validation Loss: 1.28 | Validation Acc: 53.40
Epoch 30 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 1.32 | Validation Acc: 54.60
Epoch 40 | Training Loss: 0.00 | Train Acc: 100.00 | Validation Loss: 1.35 | Validation Acc: 52.20
Epoch 50 | Training Loss: 0.00 | Train Acc: 100.00 | Validation Loss: 1.34 | Validation Acc: 50.00
Epoch 60 | Training Loss: 0.00 | Train Acc: 100.00 | Validation Loss: 1.30 | Validation Acc: 51.20
Epoch 70 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 1.27 | Validation Acc: 52.20
Epoch 80 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 1.26 | Validation Acc: 51.00
Epoch 90 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 1.26 | Validation

#Theory Related to Graph Neural Network



A linear layer from a normal neural network is written as:
![](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*vf5VwYbT2yMfFF7QdStSoQ.png)

Graph:  
![](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*66EsQqSrq51bz0P2szdbRA.png)
Also self is reachable so there should be a self loop which can be achieved by adding Identity Matrix
![](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*wx6LWcvhEYs7O86XhG91kw.png)

Now we can add this updated matrix to the linear layer:
![](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*R8ex7tRgiDucTrRx-p_EVg.png)

In GNNs it’s common to use symmetric normalization. The idea is to normalize each node’s aggregated features by the square root of its degree
![](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*BfqYJV38FD6ln-lsZaHxzw.png)

We have multiple options for normalization, for example:

![](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*sI-M9orhrz2QAvdfkscagw.png)

The symmetrically normalized (adapted) adjacency matrix is computed like this:
![](https://cdn-images-1.readmedium.com/v2/resize:fit:800/1*5tREoKEmdU3_hfLMXlgmnQ.png)



#Training Using GCN

In [19]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [22]:
for model_class in [GCN]: # later we test also with GCN (this post) and GAT (next blog post)
    results[model_class.__name__] = []
    for i in range(10):
        print(f"Training {model_class.__name__} iteration {i+1}")

        # the output_dim is the number of unique classes in the set
        model = model_class(input_dim=data.x.shape[1], hidden_dim=32, output_dim=num_labels).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

        # deal with the class imbalance
        class_weights = torch.bincount(data.y) / len(data.y)
        loss_fn = nn.CrossEntropyLoss(weight=1/class_weights).to(device)

        data = data.to(device)

        # training loop
        for epoch in range(100):
            model.train()
            optimizer.zero_grad()
            out = model(data)

            # calculate loss
            train_loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            train_loss.backward()
            optimizer.step()

            if epoch % 10 == 0:
                model.eval()
                with torch.no_grad():
                    val_loss = loss_fn(out[data.val_mask], data.y[data.val_mask])
                    val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                    print(f'Epoch {epoch} | Training Loss: {train_loss.item():.2f} | Train Acc: {acc:>5.2f} | Validation Loss: {val_loss.item():.2f} | Validation Acc: {val_acc:>5.2f}')

        # final evaluation on the test set
        model.eval()
        with torch.no_grad():
            out = model(data)
            test_loss = loss_fn(out[data.test_mask], data.y[data.test_mask])
            test_acc = accuracy(out[data.test_mask].argmax(dim=1), data.y[data.test_mask])
            print(f'{model_class.__name__} Test Loss: {test_loss.item():.2f} | Test Acc: {test_acc:>5.2f}')
            results[model_class.__name__].append([acc, val_acc, test_acc])


# print average on test set and standard deviation
for model_name, model_results in results.items():
    model_results = torch.tensor(model_results)
    print(f'{model_name} Test Accuracy: {model_results[:, 2].mean():.2f} ± {model_results[:, 2].std():.2f}')

Training GCN iteration 1
Epoch 0 | Training Loss: 1.97 | Train Acc: 15.00 | Validation Loss: 1.94 | Validation Acc: 17.40
Epoch 10 | Training Loss: 0.32 | Train Acc: 93.57 | Validation Loss: 0.98 | Validation Acc: 62.60
Epoch 20 | Training Loss: 0.04 | Train Acc: 100.00 | Validation Loss: 0.64 | Validation Acc: 79.20
Epoch 30 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 0.66 | Validation Acc: 77.40
Epoch 40 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 0.65 | Validation Acc: 77.80
Epoch 50 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 0.64 | Validation Acc: 77.20
Epoch 60 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 0.63 | Validation Acc: 77.40
Epoch 70 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 0.64 | Validation Acc: 77.20
Epoch 80 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 0.64 | Validation Acc: 76.80
Epoch 90 | Training Loss: 0.01 | Train Acc: 100.00 | Validation Loss: 0.64 | Validation

**Conclusion**:  
* MLP Test Accuracy: `54.02 ± 0.77 %`
* GCN Test Accuracy: `78.90 ± 0.33 %`



References:  
1. Pytorch [GCNConv](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html)
2. Medium [Article](https://readmedium.com/graph-neural-networks-part-1-graph-convolutional-networks-explained-9c6aaa8a406e)